In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import resample
from scipy.signal import welch

# download the data from: https://crcns.org/data-sets/hc/hc-11/about-hc-11
# Grosmark, A.D., and Buzsáki, G. (2016). Diversity in neural firing dynamics supports both rigid and learned hippocampal sequences. Science 351, 1440–1443.
# Chen, Z., Grosmark, A.D., Penagos, H., and Wilson, M.A. (2016). Uncovering representations of sleep-associated hippocampal ensemble spike activity. Sci. Rep. 6, 32193.

#for simplicity, we provided the Achilles_10252013 LFP data, that is sampled to fs = 100 Hz

#LFP data, fs=100
LFP_fs100 = np.load('../../data_untracked/Achilles_10252013_lfp_100Hz.npy') 
#zscore lfp
LFP_fs100 = (LFP_fs100 - np.mean(LFP_fs100))/np.std(LFP_fs100)

In [2]:
## using the code from: https://github.com/zhd96/pi-vae/blob/main/examples/pi-vae_rat_data.ipynb
## Zhou, D., Wei, X. Learning identifiable and interpretable latent models of high-dimensional neural activity using pi-VAE. NeurIPS 2020. https://arxiv.org/abs/2011.04798
## we thank the authors for sharing their code publicly

import numpy as np
import matplotlib.pyplot as plt
import h5py

with h5py.File('../../data_untracked/Achilles_10252013_sessInfo.mat', 'r') as f:
    ## load spike info
    spikes_times = np.array(f['sessInfo']['Spikes']['SpikeTimes'])[0];
    
    ## load location info ## all in maze
    locations = np.array(f['sessInfo']['Position']['OneDLocation'])[0];
    locations_times = np.array(f['sessInfo']['Position']['TimeStamps'])[:,0];
    
    ## load maze epoch range
    maze_epoch = np.array(f['sessInfo']['Epochs']['MazeEpoch'])[:,0];

# time in maze
time_in_maze = ((spikes_times >= maze_epoch[0])*(spikes_times <= maze_epoch[1]));
# spikes during maze epoch
spikes_times = spikes_times[time_in_maze];

# bin spikes and obtain number of time steps
bin_size = 25; 
binned_spike_times = np.array(np.floor(spikes_times*1000/bin_size), dtype='int');
n_times_steps = binned_spike_times.max() - binned_spike_times.min()+1

# bin locations
tph_binned_time = np.array(np.floor((np.arange(binned_spike_times.min(),binned_spike_times.max()+1)*bin_size/1000)*40), dtype='int');
binned_locations_times = np.array(np.floor(locations_times*1000/bin_size), dtype='int');
locations_vec = np.zeros(n_times_steps)+np.nan;

for it in range(len(binned_locations_times)):
    locations_vec[binned_locations_times[it] - binned_spike_times.min()] = locations[it];

In [3]:
#fs for below data was 100, here we downsample it to 40
c, t, _ = LFP_fs100.shape
ds = []
for i in range(c):
    lfp_temp = LFP_fs100[i,:]
    resample_rate = 2.5
    # resample
    n_samples = int(len(lfp_temp)/resample_rate)
    lfp_ds = resample(lfp_temp, n_samples)
    ds.append(lfp_ds)

LFP_fs40 = np.array(ds)
LFP_fs40 = LFP_fs40[:,:,0]


In [ ]:
print("resampled lfp shape timesteps " + str(LFP_fs40.shape[1]))

In [5]:
mazeLFP = []  
for i in range(LFP_fs40.shape[0]):
    mazeLFP.append(LFP_fs40[i][np.unique(tph_binned_time)])
mazeLFP = np.array(mazeLFP).T

In [6]:
# split data in run and no run
norun_maze = mazeLFP[np.isnan(locations_vec)]
run_maze = mazeLFP[~np.isnan(locations_vec)]

In [7]:
#save the 3 data sets
np.save('../../data_untracked/norun_maze.npy', norun_maze)
np.save('../../data_untracked/run_maze.npy', run_maze)
np.save('../../data_untracked/full_maze.npy', mazeLFP)


In [8]:
fs = 40 # Hz
nperseg = 1024
#taking the mean along the channels is fine since they're highly correlated
frequencies0, psd0 = welch(np.mean(run_maze, axis=1), fs=fs, nperseg=nperseg)  
frequencies1, psd1 = welch(np.mean(norun_maze, axis=1), fs=fs, nperseg=nperseg)  
frequencies2, psd2 = welch(np.mean(mazeLFP, axis=1), fs=fs, nperseg=nperseg) 


In [ ]:
#freq0 and freq1 subplots
plt.figure(figsize=(10, 6))
plt.semilogy(frequencies0, psd0, alpha=0.8, zorder=0, label = 'run')
plt.semilogy(frequencies1, psd1, alpha=0.8, zorder=0, label = 'rest')
plt.semilogy(frequencies2, psd2, alpha=0.8, zorder=0, label = 'whole')
plt.legend()